In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import torch

import tsvar

In [6]:
mu = torch.tensor([0.05, 0.1, 0.05, 0.05])

beta = torch.tensor([1.5, 1.5, 1.0, 1.0])

alpha = torch.tensor([
    [0.7, 0.7, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.0],
    [0.0, 0.0, 0.0, 0.7],
    [0.0, 0.7, 0.7, 0.0]
])


dim = 4
# n_params = len(mu) + len(beta) + len(A.flatten())

coeffs_true = torch.cat((mu, beta, alpha.flatten()))

In [12]:
end_time = 50000.0

wold_sim = tsvar.simulate.GrangeBuscaSimulator(mu_rates=mu, Alpha_ba=alpha, Beta_b=beta)
events = wold_sim.simulate(end_time)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]

print(f"Simulated {sum(map(len, events))} events with end time: {end_time}")
print(list(map(len, events)))

Simulated 66909 events with end time: 50000.0
[8768, 21995, 23228, 12918]


## Run MLE

In [ ]:
coeffs_true = torch.cat((mu, beta, A.flatten()))

x0 = torch.cat((
    2.0 * torch.ones(dim, dtype=torch.float),
    3.0 * torch.ones(dim, dtype=torch.float),
    0.5 * torch.ones((dim, dim), dtype=torch.float).flatten()
))
 
optimizer = torch.optim.Adam([x0], lr=0.03)

learner_mle = tsvar.learners.MLELearner(
    model=tsvar.wold_model.WoldModel(), 
    prior=tsvar.priors.GaussianPrior(
        C=100.0 * torch.ones(len(coeffs_true))), 
    optimizer=optimizer, tol=1e-5, max_iter=10000, debug=False)

learner_callback_mle = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=10)

coeffs_hat_mle = learner_mle.fit(events, end_time, x0, callback=learner_callback_mle).detach().numpy()

In [ ]:
print('Ground truth:')
print('mu:')
print(coeffs_true[:dim].numpy().round(2))
print('beta:')
print(coeffs_true[dim:2*dim].numpy().round(2))
print('alpha:')
print(np.reshape(coeffs_true[2*dim:], (dim, dim)).numpy().round(2))
print()
print('MLE estimation:')
print('mu:')
print(coeffs_hat_mle[:dim].round(2))
print('beta:')
print(coeffs_hat_mle[dim:2*dim].round(2))
print('alpha:')
print(np.reshape(coeffs_hat_mle[2*dim:], (dim, dim)).round(2))

## Run BBVI

In [8]:
from tsvar.wold_model import WoldModel

class WoldModelSimple(WoldModel):
    
    def set_beta(self, beta_b):
        self.beta = beta_b
        
    def set_data(self, events, end_time=None):
        super().set_data(events, end_time)
        self.n_params = self.dim * self.dim + self.dim
    
    def log_likelihood(self, coeffs):
        # Extract each set of parameters
        mu = coeffs[:self.dim]
        alpha = coeffs[self.dim:].reshape(self.dim, self.dim)
        # Compute the log-likelihood
        log_like = 0
        for i in range(self.dim):
            # Compute the intensity at each event
            lam_ik_arr = mu[i] + torch.sum(self.valid_mask_ikj[i] * alpha[:,i] /(self.beta.unsqueeze(0) + self.delta_ikj[i]), axis=1)
            # Add the log-intensity term
            log_like += lam_ik_arr[:-1].log().sum()
            # Subtract the integral term
            log_like -= lam_ik_arr[0] * self.events[i][0]
            log_like -= torch.sum(lam_ik_arr[1:] * (self.events[i][1:] - self.events[i][:-1]))
        return log_like

In [13]:
from tsvar.wold_model import WoldModel
from tsvar.models import ModelVariational
from tsvar.priors import GaussianLaplacianPrior
from tsvar.posteriors import LogNormalPosterior
from tsvar.learners import VariationalInferenceLearner

# model_wold = WoldModelSimple()
# model_wold.set_beta(beta)
# n_params = len(mu) + len(A.flatten())

model_wold = WoldModel()
n_params = len(mu) + len(beta) + len(alpha.flatten())

model_var = ModelVariational(
    model=model_wold,
    prior=GaussianLaplacianPrior(
        C=torch.ones(n_params, dtype=torch.float) * 100.0,
        mask_gaus=torch.cat((
            torch.ones(2 * dim, dtype=torch.bool),
            torch.zeros(dim * dim, dtype=torch.bool)
        ))
    ),
    posterior=LogNormalPosterior(),
    n_samples=1,
)

coeffs_true = torch.cat((mu, beta, alpha.flatten()))

x0 = torch.tensor(np.hstack((
    np.random.normal(loc=0.1, scale=0.1, size=dim),
    np.random.normal(loc=0.1, scale=0.1, size=dim),
    np.random.normal(loc=0.1, scale=0.1, size=dim*dim),
    
    np.log(np.random.normal(loc=0.5, scale=0.1, size=dim)),
    np.log(np.random.normal(loc=0.5, scale=0.1, size=dim)),
    np.log(np.random.normal(loc=0.5, scale=0.1, size=dim*dim)),
)), dtype=torch.float)

print('--------- x0:')
print('alpha0:', x0[:n_params])
print('beta0:', x0[n_params:])
print()

optimizer = torch.optim.Adagrad([x0])

learner_callback = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=100)

learner = VariationalInferenceLearner(
    model=model_var, optimizer=optimizer, lr=0.1,
    tol=1e-6, 
    lr_gamma=0.9999, max_iter=10000,
    hyperparam_interval=1000, hyperparam_offset=np.inf, hyperparam_momentum=0.0,
)

coeffs_hat = learner.fit(events, end_time, x0, callback=learner_callback)

--------- x0:
alpha0: tensor([ 0.0139,  0.1437,  0.0419,  0.0163,  0.2082,  0.1733,  0.2367, -0.0714,
         0.2616, -0.0619,  0.1693,  0.0550, -0.0112,  0.1900,  0.2501,  0.1661,
         0.1042, -0.0594, -0.0040,  0.0496,  0.2440,  0.1317,  0.1629,  0.0191])
beta0: tensor([-0.5277, -0.9773, -0.9235, -0.4631, -0.8242, -0.5354, -0.6876, -0.4724,
        -0.6690, -0.8541, -0.8499, -0.4657, -0.6525, -0.8175, -1.0829, -0.5997,
        -0.9495, -0.7275, -0.4740, -0.9010, -0.7506, -0.6385, -0.8468, -0.5551])

iter: 10000 | dx: 4.1485e-04 | loss: 1.3021e+05    

In [14]:
x0[:n_params]

tensor([ 0.0139,  0.1437,  0.0419,  0.0163,  0.2082,  0.1733,  0.2367, -0.0714,
         0.2616, -0.0619,  0.1693,  0.0550, -0.0112,  0.1900,  0.2501,  0.1661,
         0.1042, -0.0594, -0.0040,  0.0496,  0.2440,  0.1317,  0.1629,  0.0191])

In [15]:
x0[n_params:]

tensor([-0.5277, -0.9773, -0.9235, -0.4631, -0.8242, -0.5354, -0.6876, -0.4724,
        -0.6690, -0.8541, -0.8499, -0.4657, -0.6525, -0.8175, -1.0829, -0.5997,
        -0.9495, -0.7275, -0.4740, -0.9010, -0.7506, -0.6385, -0.8468, -0.5551])

In [16]:
coeffs_hat[:n_params].detach().numpy().round(2)

array([-3.  , -2.34, -2.14, -2.76, -0.17,  2.35,  0.78, -0.39, -0.5 , -0.43, -1.96, -2.72, -2.36, -1.78, -0.73, -1.78, -2.68, -2.13, -1.62, -0.18, -2.88, -0.13,  0.09, -2.24], dtype=float32)

In [17]:
coeffs_hat[n_params:]

tensor([-1.6848, -1.8944, -2.2691, -1.6920, -1.6952, -1.0753, -1.6184, -1.8310,
        -2.3093, -2.1432, -1.5985, -1.5317, -1.2948, -1.2958, -1.2857, -1.2465,
        -1.4505, -1.2699, -1.2314, -2.3126, -1.4948, -2.3944, -2.7767, -1.4000],
       grad_fn=<SliceBackward>)

In [18]:
coeffs_true.detach().numpy().round(2)

array([0.05, 0.1 , 0.05, 0.01, 1.5 , 1.5 , 1.  , 1.  , 0.7 , 0.7 , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.  , 0.7 , 0.7 , 0.  ], dtype=float32)

In [22]:
coeffs_hat_mode = learner.model.posterior.mode(coeffs_hat[:n_params], coeffs_hat[n_params:]).detach().numpy().round(2)

In [25]:
np.reshape(coeffs_hat_mode[2*dim:], (dim, dim))

array([[0.6 , 0.64, 0.13, 0.06],
       [0.09, 0.16, 0.45, 0.16],
       [0.06, 0.11, 0.18, 0.83],
       [0.05, 0.87, 1.09, 0.1 ]], dtype=float32)

In [26]:
alpha

tensor([[0.7000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.7000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7000],
        [0.0000, 0.7000, 0.7000, 0.0000]])

In [20]:
learner.model.posterior.mean(coeffs_hat[:n_params], coeffs_hat[n_params:]).detach().numpy().round(2)

array([ 0.05,  0.1 ,  0.12,  0.06,  0.86, 11.14,  2.23,  0.69,  0.61,  0.66,  0.14,  0.07,  0.1 ,  0.18,  0.5 ,  0.18,  0.07,  0.12,  0.21,  0.84,  0.06,  0.88,  1.09,  0.11], dtype=float32)

In [21]:
np.sqrt(learner.model.posterior.variance(coeffs_hat[:n_params], coeffs_hat[n_params:]).detach().numpy().round(2))

array([0.     , 0.     , 0.     , 0.     , 0.17321, 3.91408, 0.44721, 0.1    , 0.     , 0.1    , 0.     , 0.     , 0.     , 0.     , 0.14142, 0.     , 0.     , 0.     , 0.     , 0.1    , 0.     , 0.1    , 0.     , 0.     ], dtype=float32)